In [1]:
# Import Dependencies
import pandas as pd 
import json
import requests
import numpy as np
import datetime
from datetime import timedelta


In [3]:
# Read the CSV file from AWS to Pandas Dataframe
url = "Resources/2018.csv"

df = pd.read_csv(url)

df.head()

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
0,2018-01-01,UA,2429,EWR,DEN,1517,1512.0,-5.0,15.0,1527.0,...,268.0,250.0,225.0,1605.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-01,UA,2427,LAS,SFO,1115,1107.0,-8.0,11.0,1118.0,...,99.0,83.0,65.0,414.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-01-01,UA,2426,SNA,DEN,1335,1330.0,-5.0,15.0,1345.0,...,134.0,126.0,106.0,846.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-01-01,UA,2425,RSW,ORD,1546,1552.0,6.0,19.0,1611.0,...,190.0,182.0,157.0,1120.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-01-01,UA,2424,ORD,ALB,630,650.0,20.0,13.0,703.0,...,112.0,106.0,83.0,723.0,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Filter the data for Tucson Airport
df1 = df.loc[df['ORIGIN'] == 'TUS']



# df1.to_csv('tus.csv')
# files.download('tus.csv')

# Remove the Unnamed columns which as no values
tus_df = df1.drop(['Unnamed: 27'], axis = 1)



In [5]:
# Fill all NaN with 0
# from google.colab import files
tus_df1 = tus_df.fillna(0)



In [6]:
import math

# def time_conv(mil_time):
#   hh_mm = []
#   for time in mil_time:
#     if math.isnan(time):
#       hh_mm.append(time)
#     elif time == 0:
#       hh_mm.append(datetime.datetime.strptime(str(int(time)), '%M').strftime('%H:%M'))  
#     elif time < 59:
#       hh_mm.append(datetime.datetime.strptime(str(int(time)), '%M').strftime('%H:%M'))  
#     else:
#       hh_mm.append(datetime.datetime.strptime(str(int(time)), '%H%M').strftime('%H:%M'))  
#   return hh_mm

def time_conv(mil_time):
  hh_mm = []
  for time in mil_time:
    if time == 2400:
      time = 0
    if math.isnan(time):
      hh_mm.append(time)
    elif time < 59:
      hh_mm.append(datetime.datetime.strptime(str(int(time)), '%M').strftime('%H:%M'))  
    else:
      hh_mm.append(datetime.datetime.strptime(str(int(time)), '%H%M').strftime('%H:%M'))  
  return hh_mm



In [7]:
# if dep_time = 0 the File was cancelled

tus_df['CRS_DEP_HM'] = time_conv(tus_df['CRS_DEP_TIME'])
tus_df['DEP_TIME_HM'] = time_conv(tus_df['DEP_TIME'])
tus_df['WHEELS_OFF_HM'] = time_conv(tus_df['WHEELS_OFF'])
tus_df['WHEELS_ON_HM'] = time_conv(tus_df['WHEELS_ON'])
tus_df['CRS_ARR_TIME_HM'] = time_conv(tus_df['CRS_ARR_TIME'])
tus_df['ARR_TIME_HM'] = time_conv(tus_df['ARR_TIME'])

# tus_df['CRS_ARR_HM'] = pd.to_datetime(tus_df['CRS_ARR_TIME'], format = '%H%M').dt.time

# tus_df1 = tus_df1[(tus_df1['ARR_TIME'] !=0) &  (tus_df1['ARR_TIME'] !=5)]
# tus_df1['ARR_TIME_HM'] = pd.to_datetime(tus_df1['ARR_TIME'], format = '%H%M').dt.time

# tus_df1 = tus_df1[(tus_df1['WHEELS_OFF'] !=0) & (tus_df1['WHEELS_OFF'] !=3)]
# tus_df1['WHEELS_OFF_HM'] = pd.to_datetime(tus_df1['WHEELS_OFF'], format = '%H%M').dt.time

tus_df
# tus_df.to_csv('tus.csv')
# files.download('tus.csv')

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,CRS_DEP_HM,DEP_TIME_HM,WHEELS_OFF_HM,WHEELS_ON_HM,CRS_ARR_TIME_HM,ARR_TIME_HM
421,2018-01-01,UA,1769,TUS,DEN,1440,1454.0,14.0,12.0,1506.0,...,NaN,NaN,NaN,NaN,14:40,14:54,15:06,16:35,16:42,16:39
1470,2018-01-01,UA,288,TUS,IAH,730,724.0,-6.0,11.0,735.0,...,NaN,NaN,NaN,NaN,07:30,07:24,07:35,10:24,10:54,10:29
1916,2018-01-01,AS,641,TUS,SEA,1258,1300.0,2.0,11.0,1311.0,...,NaN,NaN,NaN,NaN,12:58,13:00,13:11,15:01,15:19,15:08
3846,2018-01-01,EV,4403,TUS,IAH,1208,1145.0,-23.0,10.0,1155.0,...,NaN,NaN,NaN,NaN,12:08,11:45,11:55,14:44,15:36,14:58
6720,2018-01-01,OO,3012,TUS,LAX,800,757.0,-3.0,11.0,808.0,...,NaN,NaN,NaN,NaN,08:00,07:57,08:08,08:19,09:01,08:55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7212416,2018-12-31,OO,5923,TUS,DEN,815,825.0,10.0,10.0,835.0,...,NaN,NaN,NaN,NaN,08:15,08:25,08:35,10:01,10:25,10:28
7212429,2018-12-31,OO,5945,TUS,SFO,659,653.0,-6.0,15.0,708.0,...,NaN,NaN,NaN,NaN,06:59,06:53,07:08,08:03,08:25,08:09
7212747,2018-12-31,AA,137,TUS,DFW,624,615.0,-9.0,10.0,625.0,...,NaN,NaN,NaN,NaN,06:24,06:15,06:25,09:00,09:40,09:14
7213166,2018-12-31,AA,539,TUS,LAX,1159,1339.0,100.0,11.0,1350.0,...,0.0,0.0,0.0,89.0,11:59,13:39,13:50,14:05,12:40,14:09


In [8]:
# Add Day of the week

tus_df['FL_DATE'] = pd.to_datetime(tus_df['FL_DATE'], format = '%Y/%m/%d')
tus_df['day_of_week'] = tus_df['FL_DATE'].dt.day_name()
tus_df

# tus_df1.to_csv('tus.csv')
# files.download('tus.csv')



,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,CRS_DEP_HM,DEP_TIME_HM,WHEELS_OFF_HM,WHEELS_ON_HM,CRS_ARR_TIME_HM,ARR_TIME_HM,day_of_week
421,2018-01-01,UA,1769,TUS,DEN,1440,1454.0,14.0,12.0,1506.0,...,NaN,NaN,NaN,14:40,14:54,15:06,16:35,16:42,16:39,Monday
1470,2018-01-01,UA,288,TUS,IAH,730,724.0,-6.0,11.0,735.0,...,NaN,NaN,NaN,07:30,07:24,07:35,10:24,10:54,10:29,Monday
1916,2018-01-01,AS,641,TUS,SEA,1258,1300.0,2.0,11.0,1311.0,...,NaN,NaN,NaN,12:58,13:00,13:11,15:01,15:19,15:08,Monday
3846,2018-01-01,EV,4403,TUS,IAH,1208,1145.0,-23.0,10.0,1155.0,...,NaN,NaN,NaN,12:08,11:45,11:55,14:44,15:36,14:58,Monday
6720,2018-01-01,OO,3012,TUS,LAX,800,757.0,-3.0,11.0,808.0,...,NaN,NaN,NaN,08:00,07:57,08:08,08:19,09:01,08:55,Monday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7212416,2018-12-31,OO,5923,TUS,DEN,815,825.0,10.0,10.0,835.0,...,NaN,NaN,NaN,08:15,08:25,08:35,10:01,10:25,10:28,Monday
7212429,2018-12-31,OO,5945,TUS,SFO,659,653.0,-6.0,15.0,708.0,...,NaN,NaN,NaN,06:59,06:53,07:08,08:03,08:25,08:09,Monday
7212747,2018-12-31,AA,137,TUS,DFW,624,615.0,-9.0,10.0,625.0,...,NaN,NaN,NaN,06:24,06:15,06:25,09:00,09:40,09:14,Monday
7213166,2018-12-31,AA,539,TUS,LAX,1159,1339.0,100.0,11.0,1350.0,...,0.0,0.0,89.0,11:59,13:39,13:50,14:05,12:40,14:09,Monday


In [10]:
# Airport Codes
url_airport = "Resources/airports_codes.csv"

airport = pd.read_csv(url_airport)

airport.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9235 entries, 0 to 9234
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   AirportCode    9235 non-null   object
 1   CityName       9235 non-null   object
 2   WorldAreaCode  9235 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 216.6+ KB


In [11]:
merge_dest_df = pd.merge(airport, tus_df, left_on='AirportCode', right_on='DEST', how='inner')

In [12]:
merge_dest_origin_df = pd.merge(airport, merge_dest_df, left_on='AirportCode', right_on='ORIGIN', how='inner')
merge_dest_origin_df

,AirportCode_x,CityName_x,WorldAreaCode_x,AirportCode_y,CityName_y,WorldAreaCode_y,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,...,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,CRS_DEP_HM,DEP_TIME_HM,WHEELS_OFF_HM,WHEELS_ON_HM,CRS_ARR_TIME_HM,ARR_TIME_HM,day_of_week
0,TUS,"TUCSON, ARIZONA, USA",81,ATL,"ATLANTA, GEORGIA, USA",34,2018-01-01,DL,1240,TUS,...,0.0,0.0,55.0,14:53,16:00,16:27,21:18,20:29,21:24,Monday
1,TUS,"TUCSON, ARIZONA, USA",81,ATL,"ATLANTA, GEORGIA, USA",34,2018-01-01,DL,1345,TUS,...,NaN,NaN,NaN,07:10,07:05,07:15,12:10,12:39,12:14,Monday
2,TUS,"TUCSON, ARIZONA, USA",81,ATL,"ATLANTA, GEORGIA, USA",34,2018-01-02,DL,1240,TUS,...,NaN,NaN,NaN,14:53,14:46,15:03,19:54,20:29,20:04,Tuesday
3,TUS,"TUCSON, ARIZONA, USA",81,ATL,"ATLANTA, GEORGIA, USA",34,2018-01-02,DL,1345,TUS,...,NaN,NaN,NaN,07:10,07:03,07:14,12:04,12:39,12:09,Tuesday
4,TUS,"TUCSON, ARIZONA, USA",81,ATL,"ATLANTA, GEORGIA, USA",34,2018-01-03,DL,1240,TUS,...,NaN,NaN,NaN,13:40,13:35,13:47,18:42,19:13,18:52,Wednesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18218,TUS,"TUCSON, ARIZONA, USA",81,SLC,"SALT LAKE CITY, UTAH, USA",87,2018-12-30,OO,3942,TUS,...,NaN,NaN,NaN,17:29,17:35,17:48,19:25,19:34,19:30,Sunday
18219,TUS,"TUCSON, ARIZONA, USA",81,SLC,"SALT LAKE CITY, UTAH, USA",87,2018-12-30,OO,4539,TUS,...,NaN,NaN,NaN,08:05,07:53,08:20,09:57,10:12,10:10,Sunday
18220,TUS,"TUCSON, ARIZONA, USA",81,SLC,"SALT LAKE CITY, UTAH, USA",87,2018-12-30,OO,4567,TUS,...,NaN,NaN,NaN,12:17,12:11,12:23,13:59,14:20,14:03,Sunday
18221,TUS,"TUCSON, ARIZONA, USA",81,SLC,"SALT LAKE CITY, UTAH, USA",87,2018-12-31,OO,4539,TUS,...,NaN,NaN,NaN,08:10,08:01,08:21,09:46,10:15,09:53,Monday


In [13]:
# Rename Columns
rename_dest_origin_df = merge_dest_origin_df.rename(columns = {'CityName_x': 'origin_city_name',
                             'CityName_y': 'dest_city_name'})

rename_dest_origin_df

,AirportCode_x,origin_city_name,WorldAreaCode_x,AirportCode_y,dest_city_name,WorldAreaCode_y,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,...,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,CRS_DEP_HM,DEP_TIME_HM,WHEELS_OFF_HM,WHEELS_ON_HM,CRS_ARR_TIME_HM,ARR_TIME_HM,day_of_week
0,TUS,"TUCSON, ARIZONA, USA",81,ATL,"ATLANTA, GEORGIA, USA",34,2018-01-01,DL,1240,TUS,...,0.0,0.0,55.0,14:53,16:00,16:27,21:18,20:29,21:24,Monday
1,TUS,"TUCSON, ARIZONA, USA",81,ATL,"ATLANTA, GEORGIA, USA",34,2018-01-01,DL,1345,TUS,...,NaN,NaN,NaN,07:10,07:05,07:15,12:10,12:39,12:14,Monday
2,TUS,"TUCSON, ARIZONA, USA",81,ATL,"ATLANTA, GEORGIA, USA",34,2018-01-02,DL,1240,TUS,...,NaN,NaN,NaN,14:53,14:46,15:03,19:54,20:29,20:04,Tuesday
3,TUS,"TUCSON, ARIZONA, USA",81,ATL,"ATLANTA, GEORGIA, USA",34,2018-01-02,DL,1345,TUS,...,NaN,NaN,NaN,07:10,07:03,07:14,12:04,12:39,12:09,Tuesday
4,TUS,"TUCSON, ARIZONA, USA",81,ATL,"ATLANTA, GEORGIA, USA",34,2018-01-03,DL,1240,TUS,...,NaN,NaN,NaN,13:40,13:35,13:47,18:42,19:13,18:52,Wednesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18218,TUS,"TUCSON, ARIZONA, USA",81,SLC,"SALT LAKE CITY, UTAH, USA",87,2018-12-30,OO,3942,TUS,...,NaN,NaN,NaN,17:29,17:35,17:48,19:25,19:34,19:30,Sunday
18219,TUS,"TUCSON, ARIZONA, USA",81,SLC,"SALT LAKE CITY, UTAH, USA",87,2018-12-30,OO,4539,TUS,...,NaN,NaN,NaN,08:05,07:53,08:20,09:57,10:12,10:10,Sunday
18220,TUS,"TUCSON, ARIZONA, USA",81,SLC,"SALT LAKE CITY, UTAH, USA",87,2018-12-30,OO,4567,TUS,...,NaN,NaN,NaN,12:17,12:11,12:23,13:59,14:20,14:03,Sunday
18221,TUS,"TUCSON, ARIZONA, USA",81,SLC,"SALT LAKE CITY, UTAH, USA",87,2018-12-31,OO,4539,TUS,...,NaN,NaN,NaN,08:10,08:01,08:21,09:46,10:15,09:53,Monday


In [15]:
# Drop colums not needed.
tus_final_df = rename_dest_origin_df.drop(['AirportCode_x', 'WorldAreaCode_x', 'AirportCode_y', 'WorldAreaCode_y'], axis = 1)

DELAY = []

for delay in tus_final_df['ARR_DELAY']:
  if delay < 30:
    DELAY.append(0)
  else:
    DELAY.append(1)

tus_final_df['DELAY'] = DELAY



# Export the final DF
tus_final_df.to_csv('2018_TUS.csv', index = False)
# files.download('2015_TUS.csv')

tus_final_df



,origin_city_name,dest_city_name,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,...,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,CRS_DEP_HM,DEP_TIME_HM,WHEELS_OFF_HM,WHEELS_ON_HM,CRS_ARR_TIME_HM,ARR_TIME_HM,day_of_week,DELAY
0,"TUCSON, ARIZONA, USA","ATLANTA, GEORGIA, USA",2018-01-01,DL,1240,TUS,ATL,1453,1600.0,67.0,...,0.0,55.0,14:53,16:00,16:27,21:18,20:29,21:24,Monday,1
1,"TUCSON, ARIZONA, USA","ATLANTA, GEORGIA, USA",2018-01-01,DL,1345,TUS,ATL,710,705.0,-5.0,...,NaN,NaN,07:10,07:05,07:15,12:10,12:39,12:14,Monday,0
2,"TUCSON, ARIZONA, USA","ATLANTA, GEORGIA, USA",2018-01-02,DL,1240,TUS,ATL,1453,1446.0,-7.0,...,NaN,NaN,14:53,14:46,15:03,19:54,20:29,20:04,Tuesday,0
3,"TUCSON, ARIZONA, USA","ATLANTA, GEORGIA, USA",2018-01-02,DL,1345,TUS,ATL,710,703.0,-7.0,...,NaN,NaN,07:10,07:03,07:14,12:04,12:39,12:09,Tuesday,0
4,"TUCSON, ARIZONA, USA","ATLANTA, GEORGIA, USA",2018-01-03,DL,1240,TUS,ATL,1340,1335.0,-5.0,...,NaN,NaN,13:40,13:35,13:47,18:42,19:13,18:52,Wednesday,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18218,"TUCSON, ARIZONA, USA","SALT LAKE CITY, UTAH, USA",2018-12-30,OO,3942,TUS,SLC,1729,1735.0,6.0,...,NaN,NaN,17:29,17:35,17:48,19:25,19:34,19:30,Sunday,0
18219,"TUCSON, ARIZONA, USA","SALT LAKE CITY, UTAH, USA",2018-12-30,OO,4539,TUS,SLC,805,753.0,-12.0,...,NaN,NaN,08:05,07:53,08:20,09:57,10:12,10:10,Sunday,0
18220,"TUCSON, ARIZONA, USA","SALT LAKE CITY, UTAH, USA",2018-12-30,OO,4567,TUS,SLC,1217,1211.0,-6.0,...,NaN,NaN,12:17,12:11,12:23,13:59,14:20,14:03,Sunday,0
18221,"TUCSON, ARIZONA, USA","SALT LAKE CITY, UTAH, USA",2018-12-31,OO,4539,TUS,SLC,810,801.0,-9.0,...,NaN,NaN,08:10,08:01,08:21,09:46,10:15,09:53,Monday,0
